This file contains an annotated version of the python script ran on the remote server _hydra_, that beforms the full hierarchical bayesian model for our available data set.

The first block of code asks the user how many stars they would like to use from the full dataset, then how many iterations they would like the stan implementations to use: the first run does not use a decay term, making it faster to converge and also providing idealised starting values for the second half; these values are then used in the full model that incorporates an additional decay parameter.

In [ ]:
import numpy as np
import pandas as pd
import random
import pystan

# Define input parameters
nstars = int(input("How many stars would you like to sample? " ))
iters_notau = int(input("How many iterations would you like to use for model without decay? " ))
iters_tau = int(input("How many iterations would you like to use for model with decay? " ))

def fitsummary():
    n = 0
    while n < 1:
        print_fit = raw_input("Would you like to save the fit summary files?: " )
        ayes = ['yes', 'y', 'Yes', 'Y']
        noes = ['no', 'n', 'No', 'N']
        if any(print_fit == i for i in ayes):
            print_fit = True
            n = 1
        elif any(print_fit == i for i in noes):
            print_fit = False
            n = 1
        else:
            print("Invalid input. Please try again.")
    return(print_fit)
    
print_fit = fitsummary()

Using the selected number of stars, we then randomly select a sample, recording their names and asteroseismic information into variables.

In [ ]:
# Select stars for analysis
output = pd.read_csv('~/y4project/data/output_back_filesremoved.csv',
                     usecols=['kic','numax', 'numax_err'])
output = output.loc[(np.abs(output['numax'] - 75) < 30.0)].reset_index(drop=True)
IDs = [random.choice(output['kic']) for i in range(nstars)]
Numax = [(output.loc[(output['kic'] == IDs[i])]).iloc[0]['numax'] for i in range(len(IDs))]
Numax_err = [(output.loc[(output['kic'] == IDs[i])]).iloc[0]['numax_err'] for i in range(len(IDs))]

Having obtained our asteroseismic data, we need to read the induvidual oscillation modes for each star and store them into arrays for analysis by our HBM. The stars have different numbers of radial modes recorded; since equal length arrays are required in order for the model to run, we calculate extrapolated modes for any stars with insufficient data points and append them to the array. These points are allocated a very large error so Stan effectively ignores them when optimising the parameters for each star.

In [ ]:
modesID = [pd.read_csv('~/y4project/data/rgbmodes/modes_'+str(IDs[i])+'.csv', usecols=['f0', 'f0_err', 'A0'])
           for i in np.arange(0,len(IDs),1)]
lenmodes = [len(modesID[i]) for i in range(len(IDs))]
maxmodes = max(lenmodes)
arr_n = np.zeros([len(IDs),maxmodes])
arr_freq = np.zeros([len(IDs),maxmodes])
arr_freqerr = np.zeros([len(IDs),maxmodes])

dnu_avgID = []
for i in np.arange(0,len(IDs),1):
    modesID[i] = modesID[i].sort_values(by=['f0'])
    modesID[i] = modesID[i].set_index(np.arange(0,len(modesID[i]),1))
    modesID[i]['dnu'] = (modesID[i]['f0'].diff(2).shift(-1))/2
    dnu_avg = (np.mean(modesID[i]['dnu']))
    dnu_avgID.append(dnu_avg)
    
    n_min = int(modesID[i]['f0'].min() / dnu_avg)
    n = np.arange(n_min, n_min+len(modesID[i]), 1)
    modesID[i].insert(loc=0, column='n', value=n)
    
    # Loop to ensure all arrays are the same length
    if lenmodes[i] < maxmodes:
        l = lenmodes[i]
        while l < maxmodes:
            newrow = {'n': int(np.max(modesID[i]['n'])+1),
                      'f0': np.max(modesID[i]['f0'])+dnu_avgID[i],
                      'f0_err': 100000}
            modesID[i] = modesID[i].append(newrow, ignore_index=True)
            l += 1
        
    arr_n[i,:] = modesID[i]['n']
    arr_freq[i,:] = modesID[i]['f0']
    arr_freqerr[i,:] = modesID[i]['f0_err']
    
dnu_avgarr = np.asarray(dnu_avgID)

We can also define some start parameters for the model:

In [ ]:
tau = np.ones([len(IDs)])*10
phi = np.ones([len(IDs)])*1.71
G = np.ones([len(IDs)])*3.08
epsilon = np.ones([len(IDs)])*0.05
alpha = np.ones([len(IDs)])*0.01
A = np.ones([len(IDs)])*0.03

At this point we are ready to start running our model. The Stan code is read in from a compiled version of the model - dictionaries are supplied to allocate the various parameters in the model to the data we have made available. The Stan code for this first model is shown below:

```stan
functions {
    real glitch(real n, real dnu, real numax, real epsilon, real alpha, real A, real G, real phi){
        real nmax = numax/dnu - epsilon;
        return (n + epsilon + alpha/2 * (n-nmax)^2 + 
                A*G/(2*pi()) * sin((2*pi()*(n-nmax))/G + phi)) * dnu;
    }
}
data {
    int N;  //number of stars
    int M; //number of modes
    real n[N, M];
    real freq[N, M];
    real freq_err[N, M];
    real numax_obs[N];
    real numax_err[N];
    real dnu_guess[N];
}
parameters {
	// Normal Parameters
    real dnu[N];
    real numax[N];
    real<lower = -2.0*pi(), upper = 2.0*pi()> phi[N];
    real G[N];


	// Hierarchical Parameters
    real epsilon_std[N];
    real<lower=0> eps_std;
    real epsA;
    real epsB;

    real alpha_std[N];
    real<lower=0> al_std;
    real alA;
    real alB;

    real A_std[N];
    real<lower=0> A_err;
    real AA;
    real AB;
    
}

transformed parameters {
    real epsilon[N];
    real alpha[N];
    real A[N];
    for (i in 1:N){
        epsilon[i] = epsilon_std[i] * eps_std + (epsA + epsB * log(dnu[i]));
        alpha[i] = alpha_std[i] * al_std + (alA * (dnu[i])^(-alB));
        A[i] = A_std[i] * A_err + (AA * (dnu[i])^(-AB));
    }
}

model {
    real mod[M];
    for (i in 1:N){
        for (j in 1:M){
            mod[j] = glitch(n[i,j], dnu[i], numax[i], epsilon[i], alpha[i], A[i], G[i], phi[i]);
        }
        freq[i,:] ~ normal(mod, freq_err[i,:]);
        dnu[i] ~ normal(dnu_guess[i], dnu_guess[i]*0.01);
		numax[i] ~ normal(numax_obs[i], numax_err[i]);
    }
    
    G ~ normal(3.08, 0.65);
    
    // Hierarchical Parameters
    epsilon_std ~ normal(0, 1);
    eps_std ~ normal(0, 0.5);
    epsA ~ normal(0.601, 0.25);
    epsB ~ normal(0.632, 0.25);

    alpha_std ~ normal(0, 1);
    al_std ~ normal(0, 0.5);    
    alA ~ normal(0.015, 0.005);
    alB ~ normal(0.32, 0.08);

    A_std ~ normal(0, 1);
    A_err ~ normal(0, 0.5);
    AA ~ normal(0.06, 0.02);
    AB ~ normal(0.88, 0.05);
}

```

In [ ]:
import pickle
sm1 = pickle.load(open('no_tau_stan.pkl', 'rb'))

stan_data = {'N': len(IDs),
         'M': maxmodes,
         'n': arr_n, 
         'freq': arr_freq,
         'freq_err': arr_freqerr,
         'numax_obs': Numax,
         'numax_err': Numax_err,
         'dnu_guess': dnu_avgarr
        }
start = {'dnu': dnu_avgarr,
         'numax': Numax,
         'eps_std': 0.01,
         'al_std': 0.01,
         'A_err': 0.01,
         #'G_err': 0.65,
         'epsilon': epsilon,
         'alpha': alpha,
         'A': A,
         'G': G,
         'phi': phi,
         'epsA': 0.601,
         'epsB': 0.632,
         'alA': 0.015,
         'alB': 0.32,
         'AA': 0.06,
         'AB': 0.88,
         #'GA': 3.08
         #'tau': tau
    }
nchains = 4

fit1 = sm1.sampling(data=stan_data, iter=iters_notau, chains=nchains, init=[start for n in range(nchains)])

We can take the results and print out a summary file (showing statistical and convergence summary of each parameter), as well as saving the results for each star to a .csv file for later use.

In [ ]:
summ_notau = fit1.stansummary()

params = np.zeros([len(IDs), 9])

for i in np.arange(0,len(IDs),1):
    params[i] = [IDs[i],
                np.mean(fit1['dnu'], axis=0)[i],
                np.mean(fit1['numax'], axis=0)[i],
                np.mean(fit1['epsilon'], axis=0)[i],
                np.mean(fit1['alpha'], axis=0)[i],
                np.mean(fit1['A'], axis=0)[i],
                np.mean(fit1['G'], axis=0)[i],
                np.mean(fit1['phi'], axis=0)[i],
                np.nan]

np.savetxt('no_tau_models.csv', params, delimiter=',')

if print_fit == True:
    with open('fullsumm_notau.txt', 'w') as f:
        f.write(summ_notau)

Now we have a file with the optimised parameters for each star, we can read this in to provide the starting parameters for each star. We also make an additional check for any stars that have unphysical/outlying values for some of the parameters. Using the indexes of these stars, we remove the corresponding data from the arrays that would have been parsed into our second stan model.

In [ ]:
no_tau = pd.read_csv('no_tau_models.csv', names=['kic', 'dnu', 'numax',
                     'epsilon', 'alpha', 'A', 'G', 'phi', 'tau'])
idxs = no_tau[(no_tau['A']<0.0) | (no_tau['A']>0.1) |
              (no_tau['alpha']<0.0)| (no_tau['epsilon']>0.35)].index

arr_n = np.delete(arr_n, [idxs], axis=0)
arr_freq = np.delete(arr_freq, [idxs], axis=0)
arr_freqerr = np.delete(arr_freqerr, [idxs], axis=0)
Numax = np.delete(Numax, [idxs], axis=0)
Numax_err = np.delete(Numax_err, [idxs], axis=0)
dnu_avgarr = np.delete(dnu_avgarr, [idxs], axis=0)

no_tau = no_tau.drop(no_tau.index[[idxs]])

The second stan model takes the following form:

```stan
functions {
    real glitch(real n, real dnu, real numax, real epsilon, real alpha, real A, real G, real phi, real tau){
        real nmax = numax/dnu - epsilon;
        return (n + epsilon + alpha/2 * (n-nmax)^2 + 
                (A*G)/(2*pi()) * sin((2*pi()*(n-nmax))/G + phi) * exp(-(n-nmax)/tau)) * dnu;
    }
}

data {
    int N;                  //number of stars
    int M;                  //number of modes
    real n[N, M];           //radial mode number
    real freq[N, M];        //l=0 frequencies
    real freq_err[N, M];
    real numax_obs[N];
    real numax_err[N];
    real dnu_guess[N];
}

parameters {
    // Normal Parameters
    real dnu[N];
    real numax[N];
    real<lower = -2.0*pi(), upper = 2.0*pi()> phi[N];
    real G[N];
    real<lower = 0> tau[N];
    
    
    // Hierarchical Parameters
    real eps_std[N];
    real<lower=0> eps_sig;
    real epsA;
    real epsB;
    
    real al_std[N];
    real<lower=0> al_sig;
    real alA;
    real alB;
    
    real A_std[N];
    real<lower=0> A_sig;
    real AA;
    real AB;
    
}

transformed parameters {
    real epsilon[N];
    real alpha[N];
    real A[N];
    
    for (i in 1:N){
        epsilon[i] = eps_std[i] * eps_sig + (epsA + epsB * log(dnu[i]));
        alpha[i] = al_std[i] * al_sig + (alA * (dnu[i])^(-alB));
        A[i] = A_std[i] * A_sig + (AA * (dnu[i])^(-AB));
    }
}

model {
    real mod[M];
    for (i in 1:N){
        for (j in 1:M){
            mod[j] = glitch(n[i,j], dnu[i], numax[i], epsilon[i], alpha[i], A[i], G[i], phi[i], tau[i]);
        }
        freq[i,:] ~ normal(mod, freq_err[i,:]);
        dnu[i] ~ normal(dnu_guess[i], dnu_guess[i]*0.01);
        numax[i] ~ normal(numax_obs[i], numax_err[i]);
    }
    
    G ~ normal(3.08, 0.65);  
    tau ~ normal(10, 4);
    
    // Hierarchical Parameters
    eps_std ~ normal(0, 1);
    eps_sig ~ normal(0, 0.5);
    epsA ~ normal(0.601, 0.25);
    epsB ~ normal(0.632, 0.25);
    
    al_std ~ normal(0, 1);
    al_sig ~ normal(0, 0.5);    
    alA ~ normal(0.015, 0.005);
    alB ~ normal(0.32, 0.08);
    
    A_std ~ student_t(8, 0, 1);
    A_sig ~ normal(0, 0.5);
    AA ~ normal(0.06, 0.02);
    AB ~ normal(0.88, 0.05);
}

```

Which we again apply starting conditions to, before saving the final results in another set of summary and .csv files. NOTE: This last stage of modelling can take a particularly long time depending on the initial conditions, so it recommended to use `screen` when running on a remote server.

In [ ]:
sm2 = pickle.load(open('tau_stan.pkl', 'rb'))

stan_data = {'N': len(no_tau['kic']),
         'M': maxmodes,
         'n': arr_n, 
         'freq': arr_freq,
         'freq_err': arr_freqerr,
         'numax_obs': Numax,
         'numax_err': Numax_err,
         'dnu_guess': dnu_avgarr
        }
start = {'dnu': dnu_avgarr,
         'numax': Numax,
         'eps_sig': 0.01,
         'al_sig': 0.01,
         'A_sig': 0.01,
         #'G_sig': 0.65,
         'epsilon': no_tau['epsilon'],
         'alpha': no_tau['alpha'],
         'A': no_tau['A'],
         'G': no_tau['G'],
         'phi': no_tau['phi'],
         'epsA': 0.601,
         'epsB': 0.632,
         'alA': 0.015,
         'alB': 0.32,
         'AA': 0.07,
         'AB': 0.87,
         #'GA': 3.08
         'tau': np.ones([len(no_tau['kic'])])*10
    }
nchains = 4

fit2 = sm2.sampling(data=stan_data, iter=iters_tau, chains=nchains, init=[start for n in range(nchains)])
summ_tau = fit2.stansummary()

params = np.zeros([len(no_tau['kic']), 9])

for i in np.arange(0,len(no_tau['kic']),1):
    params[i] = [IDs[i],
                np.mean(fit2['dnu'], axis=0)[i],
                np.mean(fit2['numax'], axis=0)[i],
                np.mean(fit2['epsilon'], axis=0)[i],
                np.mean(fit2['alpha'], axis=0)[i],
                np.mean(fit2['A'], axis=0)[i],
                np.mean(fit2['G'], axis=0)[i],
                np.mean(fit2['phi'], axis=0)[i],
                np.mean(fit2['tau'], axis=0)[i]]

np.savetxt('tau_models.csv', params, delimiter=',')

if print_fit == True:
    with open('fullsumm_tau.txt', 'w') as f:
        f.write(summ_tau)